# Parte 1: carregando a base de dados e criando o problema.

Primeiro, vamos importar três funções: <br>
- read_documents, responsável por puxar os textos da base de dados. <br>
- print_search_result, responsável por
- run_search, responsável por executar alguma busca especificada pelo usuário. <br>

Depois, importaremos duas classes: <br>
- LanguageModel, responsável por administrar a criação do modelo de linguagem com seus n-gramas.
- CompleteSentence, que herda de LanguageModel e faz a interface com o problema de busca.

In [1]:
%load_ext autoreload
%autoreload 2
from utils import read_documents, print_search_result
from Searches import run_search
from UpperClasses import LanguageModel
from completeSentence import CompleteSentence
from completeSentence2 import CompleteSentence2

   As opções que temos de documentos são, em ordem crescente de tamanho:
- "safatle"
- "flor"
- "mmorpho"
- "saka"
- "machado"
- "todos", a agregação de todos os anteriores.


A seguir, precisamos definir um N para o nosso modelo. Note que o modelo também cria os (N-1)-grama para podermos, no futuro, calcular a probabilidade condicional de um dado N-grama.

In [2]:
N = 3
print(f"Creating a {N}-gram Language Model based on safatle document")
text1 = read_documents("safatle")
model1 = LanguageModel(text1, N)

Creating a 3-gram Language Model based on safatle document
Our vocabulary is made of approximately 14036 different words
Our model was made based on 5670 sentences
We have 116690 instances of 3-grams and 70307 instances of 2-grams


In [3]:
print(f"Creating a {N}-gram Language Model based on machado document")
text2 = read_documents("machado")
model2 = LanguageModel(text2, N)

Creating a 3-gram Language Model based on machado document
Our vocabulary is made of approximately 78208 different words
Our model was made based on 175260 sentences
We have 1828725 instances of 3-grams and 739128 instances of 2-grams


In [4]:
print(f"Creating a {N}-gram Language Model based on all documents")
text3 = read_documents("all")
model3 = LanguageModel(text3, N)

Creating a 3-gram Language Model based on all documents
Our vocabulary is made of approximately 140683 different words
Our model was made based on 353096 sentences
We have 3748964 instances of 3-grams and 1443434 instances of 2-grams


Com esses modelos, nós podemos acessar a contagem de cada N e (N-1)-gramas.

In [5]:
most_common_count = model3.n_grams.most_common(1)[0]
print(f"The most common n-gram together with its count was: {most_common_count}")
most_common = most_common_count[0]
most_common_smaller = model3.n_grams_smaller.most_common()[0][0]
print(f"In the first model, the n-gram {most_common} occurs {model1.n_grams[most_common]}, and the (n-1)-gram {most_common_smaller} occurs {model1.n_grams_smaller[most_common_smaller]}")
print(f"In the second model, the n-gram {most_common} occurs {model2.n_grams[most_common]}, and the (n-1)-gram {most_common_smaller} occurs {model2.n_grams_smaller[most_common_smaller]}")
print(f"In the third model, the n-gram {most_common} occurs {model3.n_grams[most_common]}, and the (n-1)-gram {most_common_smaller} occurs {model3.n_grams_smaller[most_common_smaller]}")

The most common n-gram together with its count was: (('<s>', '<s>', 'o'), 24565)
In the first model, the n-gram ('<s>', '<s>', 'o') occurs 345, and the (n-1)-gram ('<s>', '<s>') occurs 5670
In the second model, the n-gram ('<s>', '<s>', 'o') occurs 8882, and the (n-1)-gram ('<s>', '<s>') occurs 175260
In the third model, the n-gram ('<s>', '<s>', 'o') occurs 24565, and the (n-1)-gram ('<s>', '<s>') occurs 353096


# Parte 2: modelando como busca.

Nós iremos agora definir completar uma frase como um problema de busca. <br>
A nossa classe CompleteSentence herda de Language Model, por isso sua inicialização será semelhante ao que vimos anteriormente. <br>
Para sua inicialização precisaremos de três parâmetros: um documento (string), um N (int), e uma sentença (string). 

In [6]:
cs = CompleteSentence(text3, N, "<s> <s> <s>")

Our vocabulary is made of approximately 140683 different words
Our model was made based on 353096 sentences
We have 3748964 instances of 3-grams and 1443434 instances of 2-grams


A classe CompleteSentence também herda da classe Problem vista em aula. <br>
Existem 5 funções herdadas que devem ser implementadas. <br>
A primeira é a initialState(), que deve retornar um estado baseado na frase inicial passada no momento da instanciação do objeto.

In [7]:
print(f"The initial state of the problem is: {cs.initialState()}")

The initial state of the problem is: ('<s>', '<s>', '<s>')


A segunda função é a actions(state), a qual deve retornar todas as ações que podem ser efetuadas sobre state.

In [8]:
initial = cs.initialState()
options = cs.actions(initial)
print(f"The initial state has a branching factor of {len(options)}.")

The initial state has a branching factor of 21977.


A terceira e a quarta função são a result(state, action) e cost(state1, action, state2). <br>
A função result é responsável por receber um estado1 e uma ação e retornar um estado2, resultante de aplicar a ação sobre estado1. <br>
A função cost(state1, action, state2) retorna o custo de ir do estado state1 para estado state2 através de action. <br>

In [9]:
for index, op in enumerate(options):
    v = cs.result(initial, op)
    print(f"{initial} ---'{op}'---> {v} costs {cs.cost(initial, op, v):.2f}")
    if index == 5:
        print("There may be more actions, but we are stopping here...")
        break
        

('<s>', '<s>', '<s>') ---'inocentes'---> ('<s>', '<s>', '<s>', 'inocentes') costs 12.77
('<s>', '<s>', '<s>') ---'tranqüilizado'---> ('<s>', '<s>', '<s>', 'tranqüilizado') costs 12.77
('<s>', '<s>', '<s>') ---'contos'---> ('<s>', '<s>', '<s>', 'contos') costs 11.17
('<s>', '<s>', '<s>') ---'perdoa'---> ('<s>', '<s>', '<s>', 'perdoa') costs 9.83
('<s>', '<s>', '<s>') ---'tetra'---> ('<s>', '<s>', '<s>', 'tetra') costs 12.77
('<s>', '<s>', '<s>') ---'carlotaqueria'---> ('<s>', '<s>', '<s>', 'carlotaqueria') costs 12.08
There may be more actions, but we are stopping here...


A última função é a responsável por identificação dos estados meta.
Certifique-se que a sua função isGoal(state) retorna verdadeiro se state for meta e false caso contrário.

In [10]:
fake_goal = cs.initialState()
true_goal = cs.result(cs.result(fake_goal, "."), "</s>")
assert cs.isGoal(fake_goal) == False, "Initial state is not a goal"
assert cs.isGoal(true_goal) == True, f"The state {true_goal} is a goal"

# Parte 3: Executando as buscas

Para executarmos as buscas, nós iremos utilizar a função run_search. Esta função recebe o nome de uma busca, uma instância de problema e possivelmente mais parâmetros. <br>


Primeiro, executaremos uma busca de custo uniforme. Nessa busca, cada nó tem como prioridade a probabilidade total que leva até ele. <br>
Com essa busca nós encontramos a resposta ótima. Note que ela é relativamente pequena, pois quanto maior a sentença, menos provável ela é. <br>

In [11]:
import cProfile
with cProfile.Profile() as pr:
    S = run_search("UniformCost", cs)
    print_search_result(S)

Found the goal state after visiting 380 states, its log-probability is 7.607439131158633. The queue ended with 149787 elements and its maximum size was 149788.
The solution has 4 steps
('<s>', '<s>', '<s>')
('<s>', '<s>', '<s>', 'por')
('<s>', '<s>', '<s>', 'por', 'quê')
('<s>', '<s>', '<s>', 'por', 'quê', '?')


In [12]:
import pstats
stats = pstats.Stats(pr)
stats.sort_stats(pstats.SortKey.TIME)
stats.print_stats()
stats.dump_stats(filename='profiling-cs1.prof')

         1975256 function calls in 35.279 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      379   28.835    0.076   28.835    0.076 d:\Programming\Python\IME-IA-EPs\EP-IA\completeSentence.py:49(<listcomp>)
      379    2.705    0.007    2.705    0.007 {method 'copy' of 'set' objects}
        1    2.480    2.480   35.247   35.247 d:\Programming\Python\IME-IA-EPs\EP-IA\Searches.py:5(UniformCostSearch)
   150167    0.314    0.000    0.808    0.000 d:\Software\Python310\lib\queue.py:122(put)
   150166    0.297    0.000    0.352    0.000 d:\Programming\Python\IME-IA-EPs\EP-IA\completeSentence.py:56(cost)
   150547    0.084    0.000    0.185    0.000 d:\Software\Python310\lib\threading.py:359(notify)
   150547    0.076    0.000    0.117    0.000 d:\Software\Python310\lib\threading.py:264(__enter__)
   150547    0.062    0.000    0.101    0.000 d:\Software\Python310\lib\threading.py:279(_is_owned)
   150167    0.060    0.000  

Nós podemos tentar gerar frases mais longas fazendo uma busca em profundidade limitada. <br>
As possíveis vantagens de utilizar essa busca, além de gerar uma frase mais longa, é utilizar menos memória. <br>
A desvantagem é que não garantimos mais que encontraremos a resposta ótima. <br>

In [13]:
for i in range(4, 7):
    print("Maximum depth: ", i)
    S2 = run_search("LimitedDepthSearch", cs, i)
    print_search_result(S2)

Maximum depth:  4
Found the goal state after visiting 28117 states, its log-probability is 11.41470895111154. The queue ended with 26953 elements and its maximum size was 30292.
The solution has 4 steps
('<s>', '<s>', '<s>')
('<s>', '<s>', '<s>', 'o')
('<s>', '<s>', '<s>', 'o', 'que')
('<s>', '<s>', '<s>', 'o', 'que', '?')
Maximum depth:  5
Found the goal state after visiting 5295 states, its log-probability is 10.903789143366282. The queue ended with 28005 elements and its maximum size was 29201.
The solution has 5 steps
('<s>', '<s>', '<s>')
('<s>', '<s>', '<s>', 'o')
('<s>', '<s>', '<s>', 'o', 'que')
('<s>', '<s>', '<s>', 'o', 'que', 'é')
('<s>', '<s>', '<s>', 'o', 'que', 'é', '?')
Maximum depth:  6
Found the goal state after visiting 18277 states, its log-probability is 16.894266646626637. The queue ended with 29067 elements and its maximum size was 30650.
The solution has 6 steps
('<s>', '<s>', '<s>')
('<s>', '<s>', '<s>', 'o')
('<s>', '<s>', '<s>', 'o', 'que')
('<s>', '<s>', '<s>

Um método comum para geração de sentenças é o Beam Search. <br>
Esse algoritmo é um tipo de busca local e segue o pseudo-código à seguir: <br>

Esse método é mais rápido que os anteriores e gera candidatas diferentes de resposta. <br>
Suas desvantagens são que ele não possui garantias de encontrar a resposta ótima e nem que ele vai encontrar alguma resposta. <br>
Como as respostas geradas são de tamanho diferente, a métrica utilizada para compará-las não é a probabilidade, mas sim a perplexidade. <br>
Perplexidade de uma sentença é definida como o -log da probabilidade dessa sentença dividida pelo tamanho da sentença. <br>

In [14]:
S3 = run_search("BeamSearch", cs, 15)

One goal found, there are 14 left
One goal found, there are 13 left
One goal found, there are 12 left
One goal found, there are 11 left
One goal found, there are 10 left
Iteracao  10
One goal found, there are 9 left
One goal found, there are 8 left
One goal found, there are 7 left
One goal found, there are 6 left
One goal found, there are 5 left
One goal found, there are 4 left
One goal found, there are 3 left
One goal found, there are 2 left
One goal found, there are 1 left
One goal found, there are 0 left


O Beam Search retorna uma lista de triplas (histórico, probabilidade, perplexidade), onde histórico contém todos os estados que foram utilizados até chegar naquele estado meta. <br>
Como a definição de estado varia de pessoa para pessoa, fica a cargo de vocês completar a função para que a sentença final seja apresentada corretamente. <br>

In [15]:
def print_beam_search(solutions):
    """
    Change this function so that it prints the proper sentence from h
    """
    for solution in solutions:
        h, probability, perplexity = solution
        # já que nosso estado é a frase parcial, basta imprimir o estado terminal
        sentence = ""
        final_state = h[-1]
        for word in final_state:
            sentence += word+" "
        print(f"The sentence: {sentence} has probability {probability:.2f}, size {len(h)}, thus {perplexity:.2f} of perplexity")

In [16]:
print_beam_search(S3)

The sentence: <s> <s> <s> não , senhor . </s>  has probability 10.20, size 6, thus 1.70 of perplexity
The sentence: <s> <s> <s> de acordo com a mesma coisa . </s>  has probability 13.88, size 9, thus 1.54 of perplexity
The sentence: <s> <s> <s> de acordo com nota divulgada à imprensa . </s>  has probability 14.57, size 10, thus 1.46 of perplexity
The sentence: <s> <s> <s> de acordo com nota divulgada pelo mpf . </s>  has probability 14.82, size 10, thus 1.48 of perplexity
The sentence: <s> <s> <s> de acordo com dados do ministério do trabalho . </s>  has probability 15.71, size 11, thus 1.43 of perplexity
The sentence: <s> <s> <s> de acordo com nota divulgada pelo ministério do trabalho . </s>  has probability 16.26, size 12, thus 1.35 of perplexity
The sentence: <s> <s> <s> de acordo com dados do ministério do trabalho escravo . </s>  has probability 16.60, size 12, thus 1.38 of perplexity
The sentence: <s> <s> <s> de acordo com dados do ministério público do trabalho . </s>  has prob

Note que agora a função deixou de ser monotônica: ao expandirmos um nó, os filhos dele podem ter custo menor que do filho.

# Parte 4: Mudando o problema

Implemente a classe CompleteSentencePerplexity para que agora possamos executar um problema de busca utilizando a perplexidade ao invés da probabilidade. <br>

O resultado das próximas duas células deve ser exatamente igual ao do anterior.

In [17]:
cs2 = CompleteSentence2(text3, N, "<s> <s> <s>")

Our vocabulary is made of approximately 140683 different words
Our model was made based on 353096 sentences
We have 3748964 instances of 3-grams and 1443434 instances of 2-grams


In [18]:
cs2.initialState()

('<s>', '<s>', '<s>')

In [19]:
initial = cs2.initialState()
options = cs2.actions(initial)
print(f"The initial state has a branching factor of {len(options)}.")

The initial state has a branching factor of 21977.


O resultado da próxima célula deve ser como o da vez anterior, mas com todos os custos divididos pela mesma constante.

In [20]:
for index, op in enumerate(options):
    v = cs2.result(initial, op)
    print(f"{initial} ---'{op}'---> {v} costs {cs2.cost(initial, op, v):.2f}")
    if index == 5:
        print("There may be more actions, but we are stopping here...")
        break

('<s>', '<s>', '<s>') ---'inocentes'---> ('<s>', '<s>', '<s>', 'inocentes') costs 3.19
('<s>', '<s>', '<s>') ---'tranqüilizado'---> ('<s>', '<s>', '<s>', 'tranqüilizado') costs 3.19
('<s>', '<s>', '<s>') ---'contos'---> ('<s>', '<s>', '<s>', 'contos') costs 2.79
('<s>', '<s>', '<s>') ---'perdoa'---> ('<s>', '<s>', '<s>', 'perdoa') costs 2.46
('<s>', '<s>', '<s>') ---'tetra'---> ('<s>', '<s>', '<s>', 'tetra') costs 3.19
('<s>', '<s>', '<s>') ---'carlotaqueria'---> ('<s>', '<s>', '<s>', 'carlotaqueria') costs 3.02
There may be more actions, but we are stopping here...


A próxima celula deve continuar dando os mesmos resultados do anterior.

In [21]:
fake_goal = cs2.initialState()
true_goal = cs2.result(cs2.result(fake_goal, "."), "</s>")
assert cs2.isGoal(fake_goal) == False, "Initial state is not a goal"
assert cs2.isGoal(true_goal) == True, f"The state {true_goal} is a goal"

# Parte 5: reexecutando as buscas

Com essa nossa nova função de custo, a busca por custo uniforme não mais garante resposta ótima, já que uma alternativa ruim pode gerar descendentes melhores.

In [22]:
import cProfile
with cProfile.Profile() as pr:
    S4 = run_search("UniformCost", cs2)
    print_search_result(S4)

Found the goal state after visiting 117 states, its log-probability is 1.1977831338741085. The queue ended with 148625 elements and its maximum size was 148626.
The solution has 5 steps
('<s>', '<s>', '<s>')
('<s>', '<s>', '<s>', 'ao')
('<s>', '<s>', '<s>', 'ao', 'mesmo')
('<s>', '<s>', '<s>', 'ao', 'mesmo', 'tempo')
('<s>', '<s>', '<s>', 'ao', 'mesmo', 'tempo', '.')


In [23]:
import pstats
stats = pstats.Stats(pr)
stats.sort_stats(pstats.SortKey.TIME)
stats.print_stats()
stats.dump_stats(filename='profiling-cs2.prof')

         5071679 function calls in 15.093 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      116    9.230    0.080    9.230    0.080 d:\Programming\Python\IME-IA-EPs\EP-IA\completeSentence2.py:52(<listcomp>)
   148741    1.408    0.000    2.993    0.000 d:\Programming\Python\IME-IA-EPs\EP-IA\completeSentence2.py:60(cost)
        1    1.049    1.049   15.051   15.051 d:\Programming\Python\IME-IA-EPs\EP-IA\Searches.py:5(UniformCostSearch)
      116    0.865    0.007    0.865    0.007 {method 'copy' of 'set' objects}
   297482    0.824    0.000    1.271    0.000 d:\Software\Python310\lib\site-packages\nltk\util.py:823(ngrams)
   148742    0.323    0.000    0.854    0.000 d:\Software\Python310\lib\queue.py:122(put)
   891205    0.278    0.000    0.278    0.000 {built-in method math.log}
   297482    0.152    0.000    0.170    0.000 {built-in method itertools.tee}
   297482    0.119    0.000    0.167    0.000 d:\Software\Pyth

A Busca em Profundidade Iterativa continua com todas as mesmas propriedades, já que todos os elementos são comparados pelo nível que estão na árvore de busca.

In [24]:
for i in range(4, 7):
    print("Maximum depth: ", i)
    S5 = run_search("LimitedDepthSearch", cs2, i)
    print_search_result(S5)

Maximum depth:  4
Found the goal state after visiting 28117 states, its log-probability is 1.6306727073016487. The queue ended with 26953 elements and its maximum size was 30292.
The solution has 4 steps
('<s>', '<s>', '<s>')
('<s>', '<s>', '<s>', 'o')
('<s>', '<s>', '<s>', 'o', 'que')
('<s>', '<s>', '<s>', 'o', 'que', '?')
Maximum depth:  5
Found the goal state after visiting 5295 states, its log-probability is 1.3629736429207853. The queue ended with 28005 elements and its maximum size was 29201.
The solution has 5 steps
('<s>', '<s>', '<s>')
('<s>', '<s>', '<s>', 'o')
('<s>', '<s>', '<s>', 'o', 'que')
('<s>', '<s>', '<s>', 'o', 'que', 'é')
('<s>', '<s>', '<s>', 'o', 'que', 'é', '?')
Maximum depth:  6
Found the goal state after visiting 18277 states, its log-probability is 1.8771407385140708. The queue ended with 29067 elements and its maximum size was 30650.
The solution has 6 steps
('<s>', '<s>', '<s>')
('<s>', '<s>', '<s>', 'o')
('<s>', '<s>', '<s>', 'o', 'que')
('<s>', '<s>', '<s

O Beam Search também continua encontrando as mesmas respostas que anteriormente, pois em toda iteração todas as candidatas possuem mesmo tamanho.

In [25]:
S = run_search("BeamSearch", cs2, 15)

One goal found, there are 14 left
One goal found, there are 13 left
One goal found, there are 12 left
One goal found, there are 11 left
One goal found, there are 10 left
Iteracao  10
One goal found, there are 9 left
One goal found, there are 8 left
One goal found, there are 7 left
One goal found, there are 6 left
One goal found, there are 5 left
One goal found, there are 4 left
One goal found, there are 3 left
One goal found, there are 2 left
One goal found, there are 1 left
One goal found, there are 0 left


In [26]:
def print_beam_search2(solutions):
    """
    Change this function so that it prints the proper sentence from h
    """
    for solution in solutions:
        h, probability, perplexity = solution
        sentence = ""
        # já que nosso estado é a frase parcial, basta imprimir o estado terminal
        final_state = h[-1]
        for word in final_state:
            sentence += word+" "
        print(f"The sentence: {sentence} has probability {probability:.2f}, size {len(h)}, thus {perplexity:.2f} of perplexity")

In [27]:
print_beam_search2(S)

The sentence: <s> <s> <s> não , senhor . </s>  has probability 1.27, size 6, thus 0.21 of perplexity
The sentence: <s> <s> <s> de acordo com a mesma coisa . </s>  has probability 1.26, size 9, thus 0.14 of perplexity
The sentence: <s> <s> <s> de acordo com nota divulgada à imprensa . </s>  has probability 1.21, size 10, thus 0.12 of perplexity
The sentence: <s> <s> <s> de acordo com nota divulgada pelo mpf . </s>  has probability 1.24, size 10, thus 0.12 of perplexity
The sentence: <s> <s> <s> de acordo com dados do ministério do trabalho . </s>  has probability 1.21, size 11, thus 0.11 of perplexity
The sentence: <s> <s> <s> de acordo com nota divulgada pelo ministério do trabalho . </s>  has probability 1.16, size 12, thus 0.10 of perplexity
The sentence: <s> <s> <s> de acordo com dados do ministério do trabalho escravo . </s>  has probability 1.19, size 12, thus 0.10 of perplexity
The sentence: <s> <s> <s> de acordo com dados do ministério público do trabalho . </s>  has probability